In [23]:
import numpy as np 
import pandas as pd 
import random
import requests
from biopandas.pdb import PandasPdb
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy.stats import rankdata
from tqdm.notebook import tqdm
tqdm.pandas()

# Files management
import os

# Ploting
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Libraris for Data process
#import blosum as bl
import Levenshtein
from Levenshtein import distance as levenshtein_distance
from Bio.SubsMat import MatrixInfo


/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [24]:
#helper function
def read_list_from_file(list_file):
    with open(list_file) as f:
        lines  = f.readlines()
    return lines


In [25]:
curr_dir = os.getcwd()
test_df = pd.read_csv(curr_dir + "/input/deepDDG_data/novoesp-deepddg-server-predictions-sub-only/test.more.csv")
ddg = read_list_from_file(curr_dir + "/input/deepDDG_data/novoesp-deepddg-server-predictions-sub-only/wildtype_structure_prediction_af2.deepddg.ddg.txt")


In [26]:
header = ddg[0]
data = [s.split() for s in ddg[1:]]
data


[['A', 'V', '1', 'A', '-0.119'],
 ['A', 'V', '1', 'R', '-0.132'],
 ['A', 'V', '1', 'N', '-0.155'],
 ['A', 'V', '1', 'D', '-0.158'],
 ['A', 'V', '1', 'C', '-0.162'],
 ['A', 'V', '1', 'Q', '-0.111'],
 ['A', 'V', '1', 'E', '-0.147'],
 ['A', 'V', '1', 'G', '-0.196'],
 ['A', 'V', '1', 'H', '-0.075'],
 ['A', 'V', '1', 'I', '-0.156'],
 ['A', 'V', '1', 'L', '-0.200'],
 ['A', 'V', '1', 'K', '-0.138'],
 ['A', 'V', '1', 'M', '-0.155'],
 ['A', 'V', '1', 'F', '-0.209'],
 ['A', 'V', '1', 'P', '-0.162'],
 ['A', 'V', '1', 'S', '-0.245'],
 ['A', 'V', '1', 'T', '-0.287'],
 ['A', 'V', '1', 'W', '-0.096'],
 ['A', 'V', '1', 'Y', '-0.123'],
 ['A', 'P', '2', 'A', '-0.934'],
 ['A', 'P', '2', 'R', '-0.646'],
 ['A', 'P', '2', 'N', '-0.562'],
 ['A', 'P', '2', 'D', '-0.729'],
 ['A', 'P', '2', 'C', '-0.497'],
 ['A', 'P', '2', 'Q', '-0.456'],
 ['A', 'P', '2', 'E', '-0.441'],
 ['A', 'P', '2', 'G', '-0.670'],
 ['A', 'P', '2', 'H', '-0.249'],
 ['A', 'P', '2', 'I', '-0.603'],
 ['A', 'P', '2', 'L', '-0.575'],
 ['A', 'P'

In [27]:
df = pd.DataFrame(data, columns = ['chain', 'WT', 'ResID', 'Mut', 'ddG'])
df.ddG = df.ddG.astype(np.float32)
df.ResID = df.ResID.astype(int)  
df.loc[:,'location'] = df.ResID -1  #change to 0-indexing

In [28]:
test_df.location = test_df.location.fillna(-1)
test_df.location = test_df.location.astype(int)

In [29]:
#generate submission csv 
if 1:
    df.loc[:,'mut_string'] = df.WT+df.location.astype(str)+df.Mut
    test_df.loc[:,'mut_string'] =  test_df.wild_type+test_df.location.astype(str)+test_df.mutation
    
    test_df = test_df.merge(df[['ddG','mut_string']], on='mut_string',how='left')
    submit_df = pd.DataFrame({
        'seq_id': test_df.seq_id.values,
        'tm': test_df.ddG.values,
    })
    submit_df.tm = submit_df.tm.fillna(0)
    submit_df.to_csv(curr_dir + '/output/deepddg_submit.csv', index=False) #lb0.335

submit_df

,seq_id,tm
0,31390,-0.226
1,31391,-0.169
2,31392,0.000
3,31393,-1.277
4,31394,-1.353
...,...,...
2408,33798,-0.166
2409,33799,-0.064
2410,33800,-0.347
2411,33801,-0.494
